In [1]:
%pip install --upgrade py2neo

Note: you may need to restart the kernel to use updated packages.


In [1]:
from py2neo import Graph
from py2neo import Node, Relationship
import json

In [2]:
def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [3]:
password = 'k9mailneo4j'
graph = Graph('bolt://localhost:7687', auth=('neo4j', password))

In [4]:
# Delete everything in the DB
# **Warning:** only for starting over in case of inconsistent DB content.
graph.delete_all()

In [5]:
file_path = '../sample_output/k9mail.json'
data = read_json_file(file_path)

In [6]:
data['elements']['nodes'][0]

{'data': {'id': 'com.fsck.k9.Throttle.isCallbackScheduled()',
  'properties': {'sourceText': 'private boolean isCallbackScheduled() {\r\n    return runningTimerTask != null;\r\n}',
   'visibility': 'private',
   'simpleName': 'isCallbackScheduled()',
   'kind': 'method',
   'metaSrc': 'source code'},
  'labels': ['Operation']}}

In [7]:
node_dict = {node['data']['id']:Node(*node['data']['labels'], name=node['data']['properties']['simpleName'], **node['data']['properties'], qualifiedName=node['data']['id']) for node in data['elements']['nodes']}
list(node_dict.values())[0]

Node('Operation', kind='method', metaSrc='source code', name='isCallbackScheduled()', qualifiedName='com.fsck.k9.Throttle.isCallbackScheduled()', simpleName='isCallbackScheduled()', sourceText='private boolean isCallbackScheduled() {\r\n    return runningTimerTask != null;\r\n}', visibility='private')

In [8]:
data['elements']['edges'][0]

{'data': {'id': 'ac7cba3e978bef0c69a74a9b1b4a1e08',
  'source': 'com.fsck.k9.notification.NotificationController.getAccountName(com.fsck.k9.Account).account',
  'label': 'type',
  'properties': {'weight': 1, 'metaSrc': 'source code'},
  'target': 'com.fsck.k9.Account'}}

In [9]:
edge_dict = {edge['data']['id']:Relationship(node_dict[edge['data']['source']], edge['data']['label'], node_dict[edge['data']['target']], **edge['data']['properties']) for edge in data['elements']['edges']}
list(edge_dict.values())[0]

type(Node('Variable', kind='parameter', metaSrc='source code', name='account', qualifiedName='com.fsck.k9.notification.NotificationController.getAccountName(com.fsck.k9.Account).account', simpleName='account'), Node('Structure', kind='class', metaSrc='source code', name='Account', qualifiedName='com.fsck.k9.Account', simpleName='Account'), metaSrc='source code', weight=1)

In [10]:
tx = graph.begin()

for node in node_dict.values():
  tx.create(node)

for edge in edge_dict.values():
  tx.create(edge)

graph.commit(tx)


In [ ]:
packages = [
    "com.fsck.k9",
    "com.fsck.k9.account",
    "com.fsck.k9.activity",
    "com.fsck.k9.activity.compose",
    "com.fsck.k9.activity.loader",
    "com.fsck.k9.activity.misc",
    "com.fsck.k9.activity.setup",
    "com.fsck.k9.autocrypt",
    "com.fsck.k9.cache",
    "com.fsck.k9.controller",
    "com.fsck.k9.crypto",
    "com.fsck.k9.fragment",
    "com.fsck.k9.helper",
    "com.fsck.k9.helper.jsoup",
]

In [ ]:
for package in packages:
  result = graph.run(f'MATCH (p:Container)-[:contains]->(c:Structure)-[:hasScript]->(m:Operation) WHERE p.qualifiedName="{package}" m.visibility="public" AND m.kind="method" RETURN c.simpleName, c.kind, m.simpleName, m.sourceText')
  print(result)


c
"(_13:Structure {kind: 'interface', metaSrc: 'source code', name: 'FieldExtractor', qualifiedName: 'com.fsck.k9.provider.MessageProvider$FieldExtractor', simpleName: 'FieldExtractor'})"
"(_35:Structure {kind: 'class', metaSrc: 'source code', name: 'MessageProvider', qualifiedName: 'com.fsck.k9.provider.MessageProvider', simpleName: 'MessageProvider'})"
"(_67:Structure {kind: 'interface', metaSrc: 'source code', name: 'OnCryptoClickListener', qualifiedName: 'com.fsck.k9.ui.messageview.OnCryptoClickListener', simpleName: 'OnCryptoClickListener'})"
